In [ ]:
import os
import sys
import json

In [ ]:
import torch

In [ ]:
!nvcc --version

In [ ]:
!pip install detectron2 -f \
  https://dl.fbaipublicfiles.com/detectron2/wheels/cu110/torch1.7/index.html

In [ ]:
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import matplotlib.pyplot as plt
import numpy as np
import cv2
from detectron2 import model_zoo

# import some common detectron2 utilities
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

In [ ]:
MetadataCatalog.get("cvpr_train").set(thing_classes=["text"])

In [ ]:
from detectron2.engine import DefaultTrainer

cfg = get_cfg()

cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_50_FPN_3x.yaml"))
cfg.DATASETS.TRAIN = ()
cfg.DATASETS.TEST = ("cvpr_train",)
cfg.MODEL.WEIGHTS = os.path.join("/kaggle/input/faster-rcnn-text-detector/output", "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.6   # set a custom testing threshold
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 1
predictor = DefaultPredictor(cfg)

In [ ]:
import pandas as pd
data_annotations=pd.read_csv("/kaggle/input/cvpr-pricing-challenge/annotations.csv")
unique_images=list(data_annotations['img_name'].unique())
unique_images=['0000.jpg','0001.jpg','0002.jpg']

In [ ]:
import random
testing_indexes=list(range(len(unique_images)))
from detectron2.utils.visualizer import ColorMode
fig, axes = plt.subplots(3, 1, figsize=(60, 60))
axes = axes.flatten()
for index, anom_ind in enumerate(testing_indexes[:3]):
    ax = axes[index]
    d = unique_images[anom_ind]
    img = cv2.imread("/kaggle/input/cvpr-pricing-challenge/images/"+d)
    outputs = predictor(img)
    v = Visualizer(img[:, :, ::-1],
                   MetadataCatalog.get(cfg.DATASETS.TEST[0]),
                   scale=1.2,
                   instance_mode=ColorMode.IMAGE_BW
    )
    out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    ax.imshow(out.get_image()[:, :, ::-1])

In [ ]:
# pred_boxes_pickle_cvpr={}
# for i in unique_images:
#     img = cv2.imread("/kaggle/input/cvpr-pricing-challenge/images/"+i)
#     outputs = predictor(img)
#     pred_boxes_pickle_cvpr[i]=outputs["instances"].pred_boxes.tensor.tolist()

In [ ]:
# import pickle
# with open('cvpr_text_bbox.pkl', 'wb') as handle:
#     pickle.dump(pred_boxes_pickle_cvpr, handle, protocol=pickle.HIGHEST_PROTOCOL)